In [1]:
# Notebook works on Warren Buffet letters to StockHolders till 2021 and creates 
# a model out of the text data

In [2]:
!pip install transformers==4.28.0 accelerate langchain > /dev/null

In [3]:
#creating dataset by reading the files

import glob

file_names = glob.glob("/kaggle/input/warren-buffet-letters-to-investors-1977-2021/Warren_buffet_letters/Warren_buffet_letters/Chairman Letter - *.txt")

In [4]:
print(f"Number of files: {len(file_names)}")

Number of files: 45


In [5]:
text_snippets = []

for file in file_names:
    
    #print(file)
    
    try:
        print('try leg')
        with open(file,'r',encoding='cp1252') as dat:
            temp = dat.read()
    except Exception as e:
        print('Except leg')
        with open(file,'r',encoding='utf-8') as dat:
            temp = dat.read()
        
    text_snippets.append(temp)

try leg
Except leg
try leg
try leg
try leg
try leg
Except leg
try leg
try leg
Except leg
try leg
try leg
try leg
try leg
try leg
Except leg
try leg
try leg
Except leg
try leg
Except leg
try leg
Except leg
try leg
try leg
try leg
Except leg
try leg
try leg
try leg
Except leg
try leg
Except leg
try leg
Except leg
try leg
try leg
try leg
Except leg
try leg
Except leg
try leg
try leg
try leg
try leg
try leg
Except leg
try leg
try leg
try leg
Except leg
try leg
try leg
Except leg
try leg
Except leg
try leg
Except leg
try leg
try leg
Except leg
try leg
try leg
try leg
Except leg


In [6]:
import gc
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
torch.manual_seed(42)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', 
                                          bos_token='<|startoftext|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1024)

In [9]:
token_try = tokenizer.encode(text_snippets[0])

len(token_try)

Token indices sequence length is longer than the specified maximum sequence length for this model (32084 > 1024). Running this sequence through the model will result in indexing errors


32084

The input text data is crossing the sequence length. Need to truncate the sentence to accomodate all the data.

In [10]:
tokenizer.decode(token_try)

'Berkshire’s Performance vs. the S&P 500\nAnnual Percentage Change\nYear in Per-ShareBook Value ofBerkshirein Per-ShareMarket Value ofBerkshire in S&P 500with DividendsIncluded\n1965...................................................................... 23.8 49.5 10.0 1966...................................................................... 20.3 (3.4) (11.7) 1967...................................................................... 11.0 13.3 30.9 1968...................................................................... 19.0 77.8 11.0 1969...................................................................... 16.2 19.4 (8.4) 1970...................................................................... 12.0 (4.6) 3.9 1971...................................................................... 16.4 80.5 14.6 1972...................................................................... 21.7 8.1 18.9 1973...................................................................... 4. 7 (2.5) (14.8) 1974..

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
chunk_size = 1200,
chunk_overlap = 125,
length_function = len,
add_start_index = True)

text_to_seqlen = []

for snip in text_snippets:
    split_text = splitter.split_text(snip)
    text_to_seqlen.extend(split_text)

In [12]:
max_len = max([len(text) for text in text_to_seqlen])

max_len

1199

In [13]:
max_seq_len = max([len(tokenizer.encode(text)) for text in text_to_seqlen])
max_seq_len

816

In [14]:
text_to_seqlen[0:10]

['Berkshire’s Performance vs. the S&P 500\nAnnual Percentage Change\nYear in Per-ShareBook Value ofBerkshirein Per-ShareMarket Value ofBerkshire in S&P 500with DividendsIncluded',
 '1965...................................................................... 23.8 49.5 10.0 1966...................................................................... 20.3 (3.4) (11.7) 1967...................................................................... 11.0 13.3 30.9 1968...................................................................... 19.0 77.8 11.0 1969...................................................................... 16.2 19.4 (8.4) 1970...................................................................... 12.0 (4.6) 3.9 1971...................................................................... 16.4 80.5 14.6 1972...................................................................... 21.7 8.1 18.9 1973...................................................................... 4 . 7 (2.5) (14.8) 1

In [15]:
len(text_to_seqlen)

3797

In [16]:
text_to_seqlen[11]

'‹Our “Powerhouse Five” – a collection of Berkshire’s largest non-insurance businesses – had a record $12.4billion of pre-tax earnings in 2014, up $1.6 billion from 2013.* The companies in this sainted group areBerkshire Hathaway Energy (formerly MidAmerican Energy), BNSF, IMC (I’ve called it Iscar in the past),Lubrizol and Marmon.Of the five, only Berkshire Hathaway Energy, then earning $393 million, was owned by us a decade ago.Subsequently we purchased another three of the five on an all-cash basis. In acquiring the fifth, BNSF, wepaid about 70% of the cost in cash and, for the remainder, issued Berkshire shares that increased thenumber outstanding by 6.1%. In other words, the $12 billion gain in annual earnings delivered Berkshire bythe five companies over the ten-year span has been accompanied by only minor dilution. That satisfies ourgoal of not simply increasing earnings, but making sure we also increase per-shareresults. If the U.S. economy continues to improve in 2015, we expe

In [17]:
question_gen = tokenizer("|<startoftext>|" + text_to_seqlen[11][:100] +"Generate 2 questions for above text",
                               return_tensors="pt").input_ids.cuda() 

In [18]:
question_output = model.generate(question_gen,
                do_sample = True,
                top_k = 30, 
                max_length = 800,
                top_p = 0.95,
                temperature = 0.1,
                num_return_sequences = 1,
               pad_token_id=tokenizer.eos_token_id)

tokenizer.decode(question_output[0], skip_special_tokens=True)

'|<startoftext>|‹Our “Powerhouse Five” – a collection of Berkshire’s largest non-insurance businesses – had a recordGenerate 2 questions for above text'

In [19]:
class TextDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.labels = []
        self.input_ids = []
        self.attn_masks = []        
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', 
                                       truncation=True, 
                                       max_length=max_length, 
                                       padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    def __len__(self): return len(self.input_ids)
    def __getitem__(self, idx): return self.input_ids[idx], self.attn_masks[idx]

In [20]:
dataset = TextDataset(text_to_seqlen, tokenizer, max_length=max_seq_len)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
gc.collect()

160

In [21]:
torch.cuda.empty_cache()

In [22]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=1,
                                  logging_steps=100,
                                  save_steps=5000,
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  warmup_steps=10,
                                  weight_decay=0.05,
                                  logging_dir='./logs',
                                  report_to = 'none')

In [23]:
Trainer(model=model,
        args=training_args,
        train_dataset = train_dataset, 
        eval_dataset = val_dataset,
        data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                      'attention_mask': torch.stack([f[1] for f in data]),
                                      'labels': torch.stack([f[0] for f in data])}).train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,3.348500
200,0.893800
300,0.941900
400,0.826200
500,0.893800
600,0.891700
700,0.844000
800,0.844900
900,0.839300
1000,0.812500


TrainOutput(global_step=3417, training_loss=0.8709957213788344, metrics={'train_runtime': 1548.5655, 'train_samples_per_second': 2.207, 'train_steps_per_second': 2.207, 'total_flos': 5057558872915968.0, 'train_loss': 0.8709957213788344, 'epoch': 1.0})

In [24]:
generated = tokenizer("<|startoftext|> Explanation of Profit and Loss", 
                      return_tensors="pt").input_ids.cuda()

In [25]:
sample_outputs = model.generate(generated,
                                do_sample = True,
                                top_k = 30, 
                                max_length = 125,
                                top_p = 0.95,
                                temperature = 0.5,
                                num_return_sequences = 1,
                               pad_token_id=tokenizer.eos_token_id)

In [26]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, 
                                              skip_special_tokens=True)))

0:  Explanation of Profit and Losses

      In this section we present the economic and financial data that Charlie and I wish you to see.  This material is presented in a manner that 
insures that you will understand the economic and financial reasoning underlying our 
investments.  The table below shows the major components of Berkshire's 
operating earnings and, where applicable, our capital gains or losses.  
                                 


In [27]:
sample_outputs = model.generate(generated,
                                do_sample = True,
                                top_k = 30, 
                                max_length = 125,
                                top_p = 0.95,
                                temperature = 0.7,
                                num_return_sequences = 1,
                               pad_token_id=tokenizer.eos_token_id)

tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

' Explanation of Profit and Loss                                                                                                                      '

In [28]:
generated = tokenizer("<|startoftext|> When the stock market is moving", 
                      return_tensors="pt").input_ids.cuda()

sample_outputs = model.generate(generated,
                                do_sample = True,
                                top_k = 30, 
                                max_length = 125,
                                top_p = 0.95,
                                temperature = 0.7,
                                num_return_sequences = 1,
                               pad_token_id=tokenizer.eos_token_id)

tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

' When the stock market is moving at a pace that would be unwise to use as a yardstick to gauge the progress of an enterprise, I think it is safe to say that its price will be more than zero. \n                                                                              '